In [ ]:
# k-means clustering
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans

c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
c:\users\joey\appdata\local\programs\python\python37\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [ ]:
NUM_VERTIPORTS = 160
NUM_VERTIHUBS = 64
MINUTES_OF_DATA = 1
USES_ADDITIONAL_VERTHUBS = False

In [ ]:
df = pd.read_csv('data/Realtime/OpsLimits/zone17_lat_long.csv')
df = df.drop(columns=['Unnamed: 0'])
df

In [ ]:
def k_means(data, num_clusters):
    kmeans = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=300, n_init=10, random_state=0).fit(data[:,[0,1]])
    labels = kmeans.labels_
    plt.scatter(data[:,0], data[:,1], c=['tab:blue' for i in labels], alpha=.5, label='Zone', s=20)
    plt.scatter(kmeans.cluster_centers_[:,0], kmeans.cluster_centers_[:,1],s=50,c='tab:orange', label='Vertiport - Centroid')
    plt.ylabel("Latitude")
    plt.xlabel("Longitude")
    plt.legend()
    plt.title("K-Means Zone Clusters")
    plt.savefig('data/Realtime/OpsLimits/zones_to_towers_' + str(NUM_VERTIHUBS) + '.png', dpi=216)
    plt.show()
    return kmeans


In [ ]:
import copy
formatted_df = copy.deepcopy(df)
formatted_df['color'] = [float(0.5) for i in range(len(formatted_df['lat']))]
formatted_df = formatted_df[['long','lat','color']].to_numpy()
kmeans = k_means(formatted_df, NUM_VERTIPORTS)

In [ ]:
def duplicate_vertihub(vertihub_to_duplicate: int, new_vertihub_number : int, num_duplications : int, vertihubs_to_vertiports, column_header : str):
    # find the vertiports associated with the original vertihub
    vertiports = []
    for idx, vertihub in enumerate(vertihubs_to_vertiports['vertihub_0']):
        if vertihub == vertihub_to_duplicate:
            vertiports.append(idx)
    
    for i in range(num_duplications):
        for vertiport in vertiports:
            vertihubs_to_vertiports[column_header + str(i+1)][vertiport] = new_vertihub_number
            vertihubs_to_vertiports['num_vertihubs'][vertiport] += 1
        new_vertihub_number += 1
    

In [ ]:
# cluster centers = tower, cluster these towers for determing rendezvous points
tower_locations = kmeans.cluster_centers_

vertihubs_to_vertiports = pd.DataFrame()
vertihubs_to_vertiports['vertiport'] = [i for i in range(NUM_VERTIPORTS)]

vertihubs_to_vertiports['long'] = copy.deepcopy(tower_locations[:,0])
vertihubs_to_vertiports['lat'] = copy.deepcopy(tower_locations[:,1])
vertihubs_to_vertiports['color'] = [.5 for i in range(len(vertihubs_to_vertiports['lat']))]

tower_locations_formatted = vertihubs_to_vertiports[['long','lat','color']].to_numpy()

vertiport_kmeans = k_means(tower_locations_formatted, NUM_VERTIHUBS)
vertihubs_to_vertiports['num_vertihubs'] = [1 for i in range(NUM_VERTIPORTS)]
vertihubs_to_vertiports['vertihub_0'] = vertiport_kmeans.labels_
vertihubs_to_vertiports.drop(columns='color', inplace=True)

num_vertihub_columns = 1 #kind of a global variable. need to use this when assigning requests to vertihubs
total_additional_vertihubs = 0  # same as above - need this for saving the request mapping csv
if USES_ADDITIONAL_VERTHUBS:
    vertihubs_to_duplicate = [(5,1), (11, 1), (1, 1), (6, 1)] # (vertihub name, num times to dupe)
    num_vertihub_columns += max([i[1] for i in vertihubs_to_duplicate])

    # prepping the columns for storing additional vertihubs
    column_header = 'vertihub_'
    for i in range(1, num_vertihub_columns): 
        curr_header = column_header + str(i)
        vertihubs_to_vertiports[curr_header] = [-1 for i in range(NUM_VERTIPORTS)]
    
    total_additional_vertihubs = sum([i[1] for i in vertihubs_to_duplicate])
    total_vertihubs = NUM_VERTIHUBS
    for vertihub, num_duplications in vertihubs_to_duplicate:
        duplicate_vertihub(vertihub, total_vertihubs, num_duplications, vertihubs_to_vertiports, column_header)
        total_vertihubs += num_duplications
        
    vertihubs_to_vertiports.to_csv(
        'data/Realtime/OpsLimits/vertiports_' + 
        str(NUM_VERTIHUBS + total_additional_vertihubs) + 
        '_additional_vhubs_' + 
        str(total_additional_vertihubs) + '.csv',
        index=False)
else:
    vertihubs_to_vertiports.to_csv('data/Realtime/OpsLimits/vertiports_' + str(NUM_VERTIHUBS) + '.csv', index=False)
# these are actually vertihubs (orange) and vertiports (blue)
vertihubs_to_vertiports

In [ ]:
# now that each zone is assigned to a vertiport, lets change our flights from zone->zone to vertiport->vertiport

# NOTE: THIS DATA SET IS NOT INCLUDED WITH THE REPOSITORY AS IT IS A LARGE FILE. Please contact me (joeymuffoletto10@gmail.com) or Dr. Natasha Neogi (natasha.a.neogi@nasa.gov)
trips_from_work = pd.read_csv('data/Realtime/OpsLimits/commute_trips_fr_work_hhincome_all_chicago_matlab.csv')
time = trips_from_work['time']



In [ ]:
import datetime


TIME_RANGE_START = datetime.time(15,0,0)
TIME_RANGE_END = datetime.time(15,MINUTES_OF_DATA, 0)

In [ ]:
# convert to datetime.time objects
trips_between_start_end = [] 
for row in trips_from_work.itertuples(index=False):
    as_datetime = datetime.datetime.fromisoformat(row[0]).time()
    if as_datetime >= TIME_RANGE_START and as_datetime <= TIME_RANGE_END:
        trips_between_start_end.append((as_datetime, row[1], row[2], 
                                        row[3], row[4]))
print(trips_between_start_end[0])
print(len(trips_between_start_end))



In [ ]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [ ]:
# for these trips, convert their longlat origin and destination coordinate to the nearest tower

tower_locations = kmeans.cluster_centers_
# print(tower_locations)
colors = ['tab:blue', 'tab:orange', 'tab:red', 'tab:green', 'tab:purple', 'tab:brown', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
color_map = {i : color for i, color in enumerate(colors)}

frame = pd.DataFrame()
frame['long'] = copy.deepcopy(tower_locations[:,0])
frame['lat'] = copy.deepcopy(tower_locations[:,1])
frame['color'] = [.5 for i in range(len(frame['lat']))]

tower_to_rendezvous = frame
tower_to_rendezvous['tower'] = [i for i in range(NUM_VERTIPORTS)] 

formatted_trips = []
min_time = datetime.datetime.combine(datetime.date.min, min(trips_between_start_end)[0])
vertiport_trip_index = [0 for i in range(NUM_VERTIPORTS)] # default to 0 (vertihub_0). 
for trip in trips_between_start_end:
    time, origin_lat, origin_long, dest_lat, dest_long = trip
    origin = min([(ll_distance(origin_lat, origin_long, other_lat, other_long), tower) 
                  for other_lat, other_long, tower in 
                  zip(tower_to_rendezvous['lat'],
                    tower_to_rendezvous['long'],
                     tower_to_rendezvous['tower']
                     )
                 ])
    destination = min([(ll_distance(dest_lat, dest_long, other_lat, other_long), tower) 
                  for other_lat, other_long, tower in 
                  zip(tower_to_rendezvous['lat'],
                    tower_to_rendezvous['long'],
                     tower_to_rendezvous['tower']
                     )
                 ])
    normalized_time_in_seconds = (datetime.datetime.combine(datetime.date.min, time) - min_time).seconds # each time step is one second
    origin_vertiport = origin[1]
    destination_vertiport = destination[1]
    origin_vertihub = vertihubs_to_vertiports['vertihub_0'][origin_vertiport] #NOTE: the origin will never be one of the "additional" vertihubs -> this doesn't matter...yet
      #NOTE: very weird. basically, later on when you use the trips, you end up dropping ones which have the same origin and destination vertihubs. Lets do this right now so that we don't end up with garbage date that we have to deal with later
    # this fix accommadates for additional vertihubs
    #NOTE: now including trips that start and end in the same vertihub (3/8/22)
#     if origin_vertihub == vertihubs_to_vertiports['vertihub_0'][destination_vertiport]:
#       continue # dont add the trip!

    # if vertihubs_to_vertiports['vertihub_' + str(vertiport_trip_index[destination_vertiport])][destination_vertiport] == -1:
    #   vertiport_trip_index[destination_vertiport] = 0
    assert(vertihubs_to_vertiports['vertihub_' + str(vertiport_trip_index[destination_vertiport])][destination_vertiport] != -1)
    destination_vertihub = vertihubs_to_vertiports['vertihub_' + str(vertiport_trip_index[destination_vertiport])][destination_vertiport]

    vertiport_trip_index[destination_vertiport] = (vertiport_trip_index[destination_vertiport] + 1)%vertihubs_to_vertiports['num_vertihubs'][destination_vertiport] # the modulo value (after the percent sign) is the number of vertihubs that this port is mapped to. Default is 1

    

    formatted_trips.append((normalized_time_in_seconds, origin_vertihub, origin_vertiport, destination_vertihub, destination_vertiport)) # seconds from start time of simulation
# print (formatted_trips)

In [ ]:
trips_df = pd.DataFrame(formatted_trips, columns=['Time', 'OriginVertihub','OriginVertiport', 'DestinationVertihub', 'DestinationVertiport'])
path = 'data/Realtime/OpsLimits/trips_' + str(NUM_VERTIHUBS+total_additional_vertihubs) + '_minutes-' + str(MINUTES_OF_DATA)
if USES_ADDITIONAL_VERTHUBS:
    trips_df.to_csv(path + '_additional_vhubs_' + str(total_additional_vertihubs) + '.csv', index=False)
else:
    trips_df.to_csv(path + '.csv', index=False)
trips_df